In [16]:
# !pip install tensorflow==2.4.1
# !pip install transformers
# !pip install pyarrow

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from math import ceil

In [2]:
def check_targets(targs):
    if targs[0] == -1:
        return 1
    else:
        return 0

In [3]:
def create_tfrecords_dataset(data, iter_num, dataset_type='train'):
#     paper_title = tf.keras.preprocessing.sequence.pad_sequences(data['paper_title_tok'].to_list(), 
#                                                                 maxlen=512, dtype='int64', 
#                                                                 padding='post', truncating='post', value=0)
    data['no_target'] = data['target_tok'].apply(check_targets)
    data = data[data['no_target']==0].copy()
    
    paper_title = tf.ragged.constant(data['paper_title_tok'].to_list())
    
    targets = tf.keras.preprocessing.sequence.pad_sequences(data['target_tok'].to_list(), maxlen=20, 
                                                            dtype='int64', padding='post', 
                                                            truncating='post', value=0)

    ds = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(paper_title)
                              tf.data.Dataset.from_tensor_slices(data['journal_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(data['doc_type_tok'].to_list()),
                              tf.data.Dataset.from_tensor_slices(targets)))
    
    serialized_features_dataset = ds.map(tf_serialize_example)
    
    filename = f"./iteration_final/funnel_transformer/tfrecords/{dataset_type}/{str(iter_num).zfill(4)}.tfrecord"
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialized_features_dataset)

In [4]:
def tf_serialize_example(f0, f1, f2, f3, f4):
    tf_string = tf.py_function(serialize_example, (f0, f1, f2, f3, f4), tf.string)
    return tf.reshape(tf_string, ())

In [5]:
def serialize_example(paper_title, paper_mask, journal, doc_type, targets):
    paper_title_list = tf.train.Int64List(value=paper_title.numpy().tolist())
    paper_mask_list = tf.train.Int64List(value=paper_mask.numpy().tolist())
    journal_list = tf.train.Int64List(value=journal.numpy().tolist())
    doc_type_list = tf.train.Int64List(value=doc_type.numpy().tolist())
    targets_list = tf.train.Int64List(value=targets.numpy().tolist())
    
    paper_title_feature = tf.train.Feature(int64_list = paper_title_list)
    paper_mask_feature = tf.train.Feature(int64_list = paper_mask_list)
    journal_feature = tf.train.Feature(int64_list = journal_list)
    doc_type_feature = tf.train.Feature(int64_list = doc_type_list)
    targets_feature = tf.train.Feature(int64_list = targets_list)
    
    features_for_example = {
        'paper_title': paper_title_feature,
        'paper_mask': paper_mask_feature,
        'journal': journal_feature,
        'doc_type': doc_type_feature,
        'targets': targets_feature
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=features_for_example))
    
    return example_proto.SerializeToString()

In [6]:
def turn_part_file_into_tfrecord(base_path, dataset_type='train'):
    file_list = [x for x in os.listdir(f"{base_path}{dataset_type}") if x.endswith('parquet')]
    file_list.sort()
    print(f"There are {len(file_list)} files for {dataset_type}")
    for i, file_name in enumerate(file_list):
        data = pd.read_parquet(f"{base_path}{dataset_type}/{file_name}")
        print(f"_____File number: {i} ({data.shape[0]} samples)")
        create_tfrecords_dataset(data, i, dataset_type)

In [7]:
base_file_path = f"./iteration_final/funnel_transformer/tokenized_data/"

#### Without padding

In [8]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'train')

There are 50 files for train
_____File number: 0 (3338130 samples)


2021-10-29 10:12:34.897836: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-29 10:12:34.927559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-29 10:16:02.996107: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


_____File number: 1 (5007355 samples)
_____File number: 2 (2926507 samples)
_____File number: 3 (4181735 samples)
_____File number: 4 (7317781 samples)
_____File number: 5 (3645974 samples)
_____File number: 6 (3338036 samples)
_____File number: 7 (3338271 samples)
_____File number: 8 (3337774 samples)
_____File number: 9 (5006225 samples)
_____File number: 10 (5007759 samples)
_____File number: 11 (3337357 samples)
_____File number: 12 (3339298 samples)
_____File number: 13 (3337688 samples)
_____File number: 14 (5007401 samples)
_____File number: 15 (5007041 samples)
_____File number: 16 (3338928 samples)
_____File number: 17 (3338785 samples)
_____File number: 18 (3338051 samples)
_____File number: 19 (5006644 samples)
_____File number: 20 (3337507 samples)
_____File number: 21 (5006929 samples)
_____File number: 22 (5007212 samples)
_____File number: 23 (3338252 samples)
_____File number: 24 (3338310 samples)
_____File number: 25 (5007678 samples)
_____File number: 26 (3338074 samp

In [9]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'val')

There are 10 files for val
_____File number: 0 (135584 samples)
_____File number: 1 (100231 samples)
_____File number: 2 (93687 samples)
_____File number: 3 (72826 samples)
_____File number: 4 (92847 samples)
_____File number: 5 (79840 samples)
_____File number: 6 (52714 samples)
_____File number: 7 (132779 samples)
_____File number: 8 (72230 samples)
_____File number: 9 (60000 samples)
CPU times: user 5min 50s, sys: 18.9 s, total: 6min 9s
Wall time: 5min 25s


In [10]:
%%time
turn_part_file_into_tfrecord(base_file_path, 'test')

There are 5 files for test
_____File number: 0 (34952 samples)
_____File number: 1 (32169 samples)
_____File number: 2 (23371 samples)
_____File number: 3 (40252 samples)
_____File number: 4 (19886 samples)
CPU times: user 1min, sys: 3.22 s, total: 1min 3s
Wall time: 55.8 s
